In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date

In [2]:
spark = SparkSession.builder \
    .appName("App") \
    .master("spark://master:7077") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.executor.cores", "3") \
    .config("spark.executor.instances", "4") \
    .config("spark.sql.shuffle.partitions", "100") \
    .getOrCreate()

spark.conf.set("spark.sql.files.ignoreCorruptFiles", "true")
spark.conf.set("spark.sql.files.ignoreMissingFiles", "true")# total across executors

In [9]:
import requests
from datetime import datetime
SPARK_UI = "http://192.168.56.102:4040"

## Von hdfs lesen

10 Wiederholungen

In [10]:
import math
import time
import numpy as np
listh10 = []

start_all = time.time()
for i in range(10):
    dfrh= spark.read.parquet("hdfs:///input/parquet")
    app_id = requests.get(f"{SPARK_UI}/api/v1/applications").json()[0]["id"]
    job = requests.get(f"{SPARK_UI}/api/v1/applications/{app_id}/jobs").json()[0]
    start = datetime.fromisoformat(job["submissionTime"].replace("GMT", "+00:00"))
    end = datetime.fromisoformat(job["completionTime"].replace("GMT", "+00:00"))
    timer = (end - start).total_seconds()
    job_info = requests.get(f"{SPARK_UI}/api/v1/applications/{app_id}/jobs").json()
    latest_job = sorted(job_info, key=lambda j: j["submissionTime"], reverse=True)[0]
    listh10.append(timer)
    print(f"Job ID: {latest_job['jobId']} braucht {timer:.2f} Sekunden")
end_all = time.time() 

print(listh10)
print(f"Zeit für die Ausführung der Zelle: {end_all - start_all:.2f} seconds")
print(f"Mittelwert aller Spark Jobs: {np.mean(listh10):.2f} seconds")
print(f"Median aller Spark Jobs: {np.median(listh10):.2f} seconds")
print(f"Sdt aller Spark Jobs: {np.std(listh10):.2f} seconds")

Job ID: 117 braucht 0.09 Sekunden
Job ID: 118 braucht 0.08 Sekunden
Job ID: 119 braucht 0.11 Sekunden
Job ID: 120 braucht 0.16 Sekunden
Job ID: 121 braucht 0.10 Sekunden
Job ID: 122 braucht 0.12 Sekunden
Job ID: 123 braucht 0.12 Sekunden
Job ID: 124 braucht 0.13 Sekunden
Job ID: 125 braucht 0.12 Sekunden
Job ID: 126 braucht 0.09 Sekunden
[0.094, 0.08, 0.114, 0.16, 0.105, 0.123, 0.119, 0.133, 0.12, 0.089]
Zeit für die Ausführung der Zelle: 2.98 seconds
Mittelwert aller Spark Jobs: 0.11 seconds
Median aller Spark Jobs: 0.12 seconds
Sdt aller Spark Jobs: 0.02 seconds


100 Wiederholungen

In [11]:
import math
import time
import numpy as np
listh100 = []

start_all = time.time()
for i in range(100):
    dfrh= spark.read.parquet("hdfs:///input/parquet")
    app_id = requests.get(f"{SPARK_UI}/api/v1/applications").json()[0]["id"]
    job = requests.get(f"{SPARK_UI}/api/v1/applications/{app_id}/jobs").json()[0]
    start = datetime.fromisoformat(job["submissionTime"].replace("GMT", "+00:00"))
    end = datetime.fromisoformat(job["completionTime"].replace("GMT", "+00:00"))
    timer = (end - start).total_seconds()
    job_info = requests.get(f"{SPARK_UI}/api/v1/applications/{app_id}/jobs").json()
    latest_job = sorted(job_info, key=lambda j: j["submissionTime"], reverse=True)[0]
    listh100.append(timer)
    print(f"Job ID: {latest_job['jobId']} took {timer:.2f} seconds")
end_all = time.time()

print(listh100)
print(f"Zeit für die Ausführung der Zelle: {end_all - start_all:.2f} seconds")
print(f"Mittelwert aller Spark Jobs: {np.mean(listh100):.2f} seconds")
print(f"Median aller Spark Jobs: {np.median(listh100):.2f} seconds")
print(f"Sdt aller Spark Jobs: {np.std(listh100):.2f} seconds")

Job ID: 127 took 0.10 seconds
Job ID: 128 took 0.11 seconds
Job ID: 129 took 0.09 seconds
Job ID: 130 took 0.09 seconds
Job ID: 131 took 0.35 seconds
Job ID: 132 took 0.08 seconds
Job ID: 133 took 0.09 seconds
Job ID: 134 took 0.07 seconds
Job ID: 135 took 0.08 seconds
Job ID: 136 took 0.06 seconds
Job ID: 137 took 0.11 seconds
Job ID: 138 took 0.13 seconds
Job ID: 139 took 0.12 seconds
Job ID: 140 took 0.13 seconds
Job ID: 141 took 0.14 seconds
Job ID: 142 took 0.11 seconds
Job ID: 143 took 0.12 seconds
Job ID: 144 took 0.09 seconds
Job ID: 145 took 0.20 seconds
Job ID: 146 took 0.13 seconds
Job ID: 147 took 0.28 seconds
Job ID: 148 took 0.09 seconds
Job ID: 149 took 0.08 seconds
Job ID: 150 took 0.13 seconds
Job ID: 151 took 0.14 seconds
Job ID: 152 took 0.13 seconds
Job ID: 153 took 0.13 seconds
Job ID: 154 took 0.17 seconds
Job ID: 155 took 0.14 seconds
Job ID: 156 took 0.14 seconds
Job ID: 157 took 0.10 seconds
Job ID: 158 took 0.14 seconds
Job ID: 159 took 0.07 seconds
Job ID: 16

## Count testen

In [6]:
dfrh= spark.read.parquet("hdfs:///input/parquet")

In [12]:
dfrh.createOrReplaceTempView("my_table")

Einfacher select und count, ohne Ausgabe

In [14]:
import math
import time
import requests
from datetime import datetime

listcount = []

start_all = time.time()
def get_all_jobs():
    app_id = requests.get(f"{SPARK_UI}/api/v1/applications").json()[0]["id"]
    job_info = requests.get(f"{SPARK_UI}/api/v1/applications/{app_id}/jobs").json()
    return job_info

for i in range(10):
    # Step 1: Snapshot of current jobs
    jobs_before = {j["jobId"] for j in get_all_jobs()}
    
    query = f"""
    SELECT gbifID, COUNT(gbifID) AS count
    FROM my_table
    WHERE gbifID IS NOT NULL
    GROUP BY gbifID
    ORDER BY count DESC
"""

# Create Spark DataFrame
    spark_df = spark.sql(query)

# Show the result in Spark (default top 20 rows)
    spark_df.count()
    
    # Step 3: Wait to ensure Spark UI is updated
    time.sleep(2)
    
    # Step 4: Get new jobs
    jobs_after = get_all_jobs()
    new_jobs = [j for j in jobs_after if j["jobId"] not in jobs_before]
    
    # Step 5: Sum duration of all new jobs
    total_duration = 0.0
    for job in new_jobs:
        start = datetime.fromisoformat(job["submissionTime"].replace("GMT", "+00:00"))
        end = datetime.fromisoformat(job["completionTime"].replace("GMT", "+00:00"))
        duration = (end - start).total_seconds()
        total_duration += duration
        print(f"  Included Job {job['jobId']} took {duration:.2f}s")

    listcount.append(total_duration)
end_all = time.time()
# Final average
print("Average time over 10 queries:", round(math.fsum(listcount)/len(listcount), 2), "seconds")
print(listcount)
print(f"Zeit für die Ausführung der Zelle: {end_all - start_all:.2f} seconds")
print(f"Mittelwert aller Spark Jobs: {np.mean(listcount):.2f} seconds")
print(f"Median aller Spark Jobs: {np.median(listcount):.2f} seconds")
print(f"Sdt aller Spark Jobs: {np.std(listcount):.2f} seconds")

  Included Job 259 took 0.10s
  Included Job 258 took 2.74s
  Included Job 257 took 5.77s


  Included Job 262 took 0.08s
  Included Job 261 took 4.77s
  Included Job 260 took 6.30s


  Included Job 265 took 0.09s
  Included Job 264 took 4.08s
  Included Job 263 took 8.66s


  Included Job 268 took 0.07s
  Included Job 267 took 5.41s
  Included Job 266 took 7.48s


  Included Job 271 took 0.07s
  Included Job 270 took 4.50s
  Included Job 269 took 9.63s


  Included Job 274 took 0.07s
  Included Job 273 took 4.38s
  Included Job 272 took 9.77s


  Included Job 277 took 0.10s
  Included Job 276 took 4.44s
  Included Job 275 took 7.82s


  Included Job 280 took 0.12s
  Included Job 279 took 4.09s
  Included Job 278 took 7.13s


  Included Job 283 took 0.06s
  Included Job 282 took 4.07s
  Included Job 281 took 7.20s


  Included Job 286 took 0.09s
  Included Job 285 took 4.52s
  Included Job 284 took 7.61s
Average time over 10 queries: 12.12 seconds
[8.619, 11.149000000000001, 12.834, 12.96, 14.207, 14.219, 12.353, 11.338, 11.328, 12.218]
Zeit für die Ausführung der Zelle: 143.56 seconds
Mittelwert aller Spark Jobs: 12.12 seconds
Median aller Spark Jobs: 12.29 seconds
Sdt aller Spark Jobs: 1.56 seconds
